In [1]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import mediapipe as mp
import cv2
import numpy as np


In [3]:
# Calculate the angle between three points using the cosine rule
def calculate_angle(a, b, c):


    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])

    radians = np.arccos(np.dot(b - a, c - b) / (np.linalg.norm(b - a) * np.linalg.norm(c - b)))
    angle = np.degrees(radians)

    return angle

In [2]:
def count_push_ups():
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    cap = cv2.VideoCapture(0)

    # cards constraints
    card_width, card_height = 150, 80
    card_x, card_y = 10, 10
    card_color = (255, 153, 13)

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        push_up_count = 0
        is_pushing_up = False
        prev_angle = None
        delta=None

        while True:
            ret, image = cap.read()

            image = cv2.flip(image, 1)

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            results = pose.process(image_rgb)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            if results.pose_landmarks is not None:
                left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
                left_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
                left_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
                right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
                right_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW]
                right_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]

                left_arm_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                right_arm_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

                # For push-up detection, we check the angle of both arms
                if prev_angle is not None:
                    delta = abs(left_arm_angle - right_arm_angle)
                    if not is_pushing_up and delta < 10 and left_arm_angle > 160 and right_arm_angle > 160:
                        is_pushing_up = True
                    elif is_pushing_up and delta > 20 and left_arm_angle < 100 and right_arm_angle < 100:
                        is_pushing_up = False

                prev_angle = delta

                if is_pushing_up:
                    push_up_count += 1

            cv2.rectangle(image, (card_x, card_y), (card_x + card_width, card_y + card_height), card_color, -1)

            # Display the text inside the card
            text_x, text_y = card_x + 10, card_y + 30
            text_line1 = 'Push-ups'
            text_line2 = f'Count: {push_up_count}'
            line_spacing = 30
            font_scale = 0.7
            font_color = (255, 255, 255)

            cv2.putText(image, text_line1, (text_x, text_y), cv2.FONT_HERSHEY_COMPLEX, font_scale, font_color, 2, cv2.LINE_AA)
            cv2.putText(image, text_line2, (text_x, text_y + line_spacing), cv2.FONT_HERSHEY_COMPLEX, font_scale, font_color, 2, cv2.LINE_AA)

            cv2.imshow('Push-up Counter', image)

            if cv2.waitKey(1) == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

    return push_up_count

In [7]:
count = count_push_ups()

In [3]:
import matplotlib.pyplot as plt
import numpy as np

# Define a range of min detection confidence values
confidence_values = np.arange(0, 8.1, 0.1)

# Initialize lists to store accuracy values
accuracy_values = []

for confidence in confidence_values:
    # Call the count_pushups() function for each confidence value
    pushup_count = count_push_ups(video_file=None, min_detection_confidence=confidence)
    
    # Calculate accuracy (you need the ground truth push-up count)
    # For this example, let's assume the ground truth is 50 push-ups
    ground_truth_pushups = 50
    accuracy = pushup_count / ground_truth_pushups
    
    accuracy_values.append(accuracy)

# Plotting the graph
plt.plot(confidence_values, accuracy_values, marker='o')
plt.xlabel('Min Detection Confidence')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Min Detection Confidence')
plt.grid(True)
plt.show()


NameError: name 'count_pushups' is not defined